In [47]:

import sys, os
sys.path.append("../")
from iotools import readgtf
from iotools.readOxford import ReadOxford
from collections import defaultdict


gtfpath = "/cbscratch/franco/datasets/gtex/gencode.v19.annotation.gtf.gz"
gene_info = readgtf.gencode_v12(gtfpath, trim=False)

In [48]:
from collections import defaultdict
gene_dict = defaultdict(list)
for g in gene_info:
    gene_dict[g.ensembl_id] = [g.chrom, g.start, g.end]

In [49]:
gtfpath = "/cbscratch/franco/datasets/gtex/gencode.v19.annotation.gtf.gz"
gene_info = readgtf.gencode_v12(gtfpath, trim=True)
gene_trim_dict = defaultdict(list)
for g in gene_info:
    gene_trim_dict[g.ensembl_id] = [g.chrom, g.start, g.end]

In [50]:
# read GTEx chr6

chrom = 6
oxf_file = "/cbscratch/franco/datasets/gtex/GTEx_450Indiv_genot_imput_info04_maf01_HWEp1E6_dbSNP135IDs_donorIDs_dosage_chr{:d}.gz".format(chrom)
# outfile  = "/cbscratch/franco/datasets/gtex/prefiltered/GTEx_450Indiv_filtered_chr{:d}.gz".format(chrom)
fam_file = "/cbscratch/franco/datasets/gtex/donor_ids.fam"

oxf = ReadOxford(oxf_file, fam_file, isdosage=True) #should be self.args.isdosage and self.args.oxf_columns
dosage = oxf.dosage
gt_donor_ids = oxf.samplenames
snpinfo = oxf.snpinfo


started reading genotype
Read 768209 snps in 450 samples.
Finished readings snps


In [51]:
# read GTEx chr6

chrom = 6
oxf_file = "/cbscratch/franco/datasets/cardiogenics/genotypes/CG_{:d}.imputed.gz".format(chrom)
fam_file = "/cbscratch/franco/datasets/cardiogenics/genotypes/CG.sample"

oxf = ReadOxford(oxf_file, fam_file, isdosage=False) #should be self.args.isdosage and self.args.oxf_columns
dosage = oxf.dosage
gt_donor_ids = oxf.samplenames
cardio_snpinfo = oxf.snpinfo


started reading genotype
Read 1061009 snps in 786 samples.
Finished readings snps


In [52]:
from collections import defaultdict
snp_dict = defaultdict(lambda: None)
for s in cardio_snpinfo:
    snp_dict[s.varid] = s.bp_pos
    
for s in snpinfo:
    if snp_dict[s.varid]:
        if snp_dict[s.varid] != s.bp_pos:
            print("SNP exists and diff pos!!")
            print(snp_dict[s.varid]," vs ",s.bp_pos)
            raise
    snp_dict[s.varid] = s.bp_pos

# Finding cis snp-gene pairs in results

In [7]:
# read matrixEQTL SNP-Gene pairs

# gtex_out_dir      = "gtex_results"
cardio_out_dir    = "cardio_results"
MatrixEQTL_outdir = os.path.join("/cbscratch/franco/", "tejaas_output/matrixEQTL_out_filteredGT_withsomecis")

# gtex_file   = os.path.join(MatrixEQTL_outdir, gtex_out_dir, "gtex_MatrixEQTL_chr{:d}.transout")
cardio_file = os.path.join(MatrixEQTL_outdir, cardio_out_dir, "cardiogenics_MatrixEQTL_chr{:d}.transout") 



chr_list   = [6]
input_file = cardio_file
read_pairs = 0
cis_snps   = 0
window     = 1e6

notfoundgenes = list()
matrix_dict = {}
for chrm in chr_list:
    print( "Reading matrixEQTL chr{:d}".format(chrm))
    results_file = os.path.join(input_file.format(chrm))
    new_outfile = results_file + ".truetrans.beforefix"
    with open(new_outfile, 'w') as outstream:
        with open(results_file) as instream:
            _ = instream.readline()
            partial    = 0
            cis_partial= 0
            for line in instream:
                arr   = line.rstrip().split("\t")
                rsid  = arr[0]
                gene  = arr[1]
                garr  = gene_trim_dict[gene]
                if len(garr) == 0:
                    notfoundgenes.append(gene)
                    continue
                snppos= snp_dict[rsid]
                if not snppos:
                    print("skip", rsid)
                    continue
                gene_chrom= garr[0]
                low_limit = garr[1] - window
                up_limit  = garr[2] + window
                if snppos > low_limit and snppos < up_limit and gene_chrom != chrm:
                    cis_snps += 1
                    cis_partial +=1
                else:
                    outstream.write(line)
                read_pairs += 1
                partial += 1
                if partial > 1e6:
                    print("Found {:d} cis SNPs out of {:d} read pairs".format(cis_partial, partial))
                    partial    = 0
                    cis_partial= 0
print("TOTAL: Found {:d} cis SNPs out of {:d} read pairs".format(cis_snps, read_pairs))

Reading matrixEQTL chr6
Found 9765 cis SNPs out of 1000001 read pairs
Found 10377 cis SNPs out of 1000001 read pairs
Found 11105 cis SNPs out of 1000001 read pairs
Found 10485 cis SNPs out of 1000001 read pairs
TOTAL: Found 42778 cis SNPs out of 4102519 read pairs


In [9]:
# many genes are in the gx matrix but are not protein coding and hence, are deleted
myset = set(notfoundgenes)
print(len(myset))

875


In [12]:
# read matrixEQTL SNP-Gene pairs

gtex_out_dir      = "gtex_wb"
# cardio_out_dir    = "cardio_results"
MatrixEQTL_outdir = os.path.join("/cbscratch/franco/", "tejaas_output/matrixEQTL_out_filteredGT_gencode")

# infile = os.path.join(MatrixEQTL_outdir, gtex_out_dir, "gtex_MatrixEQTL_chr{:d}.transout")
infile = os.path.join(MatrixEQTL_outdir, cardio_out_dir, "cardiogenics_MatrixEQTL_chr{:d}.transout") 

chr_list   = [6]
input_file = gtex_file
read_pairs = 0
cis_snps   = 0
window     = 1e6

notfoundgenes = list()
matrix_dict = {}
for chrm in chr_list:
    print( "Reading matrixEQTL chr{:d}".format(chrm))
    results_file = os.path.join(input_file.format(chrm))
    new_outfile = results_file + ".truetrans.afterfix"
    with open(new_outfile, 'w') as outstream:
        with open(results_file) as instream:
            _ = instream.readline()
            partial    = 0
            cis_partial= 0
            for line in instream:
                arr   = line.rstrip().split("\t")
                rsid  = arr[0]
                gene  = arr[1]
                garr  = gene_dict[gene]
                if len(garr) == 0:
                    notfoundgenes.append(gene)
                    continue
                snppos= snp_dict[rsid]
                if not snppos:
                    print("skip", rsid)
                    continue
                gene_chrom= garr[0]
                low_limit = garr[1] - window
                up_limit  = garr[2] + window
                if snppos > low_limit and snppos < up_limit and gene_chrom == chrm:
                    cis_snps += 1
                    cis_partial +=1
                else:
                    outstream.write(line)
                read_pairs += 1
                partial += 1
                if partial > 1e6:
                    print("Found {:d} cis SNPs out of {:d} read pairs".format(cis_partial, partial))
                    partial    = 0
                    cis_partial= 0
print("TOTAL: Found {:d} cis SNPs out of {:d} read pairs".format(cis_snps, read_pairs))

Reading matrixEQTL chr6
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
Found 0 cis SNPs out of 1000001 read pairs
TOTAL: Found 0 cis SNPs out of 10134072 read pairs


In [ ]:
# MatrixEQTL cis SNPS in matrixEQTL_out_filteredGT_gencode

# GTEx cis snps
# TOTAL: Found 0 cis SNPs out of 10134072 read pairs

# Cardiogenics
# TOTAL: Found 0 cis SNPs out of 10134072 read pairs

In [45]:
# Load tejas SNP-Gene pairs

tejaas_outdir = os.path.join("/cbscratch/franco/", "tejaas_output/output")
# gtex_out_dir  = "gtex_results"
# gtex_out_dir  = "gtex_results_cismask_mpi"
gtex_out_dir  = "cardio_results_cismask_mpi"
# gtex_out_dir  = "cardio_results_mono_us"
# gtex_out_dir  = "cardio_results_macro"
gtex_beta     = 0.01
method        = 'jpa-rr'
gtex_dir      = os.path.join(tejaas_outdir, gtex_out_dir, method,  "beta_{:n}".format(gtex_beta))
# cardio_dir  = os.path.join(tejaas_outdir, cardio_out_dir, method, "beta_{:n}".format(cardio_beta))

chr_list   = [6]
input_dir  = gtex_dir
read_pairs = 0
cis_snps   = 0
window     = 1e6
genescistrans = defaultdict(dict)
for chrm in chr_list:
    print( "Reading TEJAAS chr{:d}".format(chrm))
    dirc = os.path.join(input_dir, "chr{:d}".format(chrm))
    pths = [os.path.join(dirc,path) for path in os.listdir(dirc) if "_gene_snp_list.txt" in path]
    for pth in pths:
        partial    = 0
        cis_partial= 0
        pvals= list()
        l = open(str(pth),"r").readlines()
        for line in l[1:]:    
            arr   = line.strip().split("\t")
            gene  = arr[0]
            if not genescistrans[gene].get("cis", False):
                genescistrans[gene]["cis"] = 0
            if not genescistrans[gene].get("trans", False):
                genescistrans[gene]["trans"] = 0
            snpid = arr[1]
            garr  = gene_trim_dict[gene]
            snppos= snp_dict[snpid]
            gene_chrom= garr[0]
            low_limit = garr[1] - window
            up_limit  = garr[2] + window
            if snppos >= low_limit and snppos <= up_limit and gene_chrom == chrm:
#                 if snppos >= garr[1] and snppos <= garr[2]:
#                     print("IN", snppos, low_limit, up_limit, garr)
#                 else:
                genescistrans[gene]["cis"] += 1
                cis_snps += 1
                cis_partial +=1
            else:
                genescistrans[gene]["trans"] += 1
            read_pairs += 1
            partial += 1
        print("Found {:d} cis SNPs out of {:d} read pairs".format(cis_partial, partial))
print("TOTAL: Found {:d} cis SNPs out of {:d} read pairs".format(cis_snps, read_pairs))

Reading TEJAAS chr6
Found 2 cis SNPs out of 754 read pairs
Found 5457 cis SNPs out of 91711 read pairs
Found 13 cis SNPs out of 7543 read pairs
Found 0 cis SNPs out of 131 read pairs
Found 1 cis SNPs out of 1024 read pairs
Found 2263 cis SNPs out of 70231 read pairs
Found 9 cis SNPs out of 848 read pairs
Found 17 cis SNPs out of 17215 read pairs
Found 1 cis SNPs out of 744 read pairs
Found 0 cis SNPs out of 2031 read pairs
Found 6 cis SNPs out of 774 read pairs
TOTAL: Found 7769 cis SNPs out of 193006 read pairs


In [46]:
genescistrans

defaultdict(dict,
            {'ENSG00000116649': {'cis': 0, 'trans': 8},
             'ENSG00000054116': {'cis': 0, 'trans': 27},
             'ENSG00000116871': {'cis': 0, 'trans': 97},
             'ENSG00000132128': {'cis': 0, 'trans': 8},
             'ENSG00000186377': {'cis': 0, 'trans': 1},
             'ENSG00000163374': {'cis': 0, 'trans': 2},
             'ENSG00000163794': {'cis': 0, 'trans': 9},
             'ENSG00000119801': {'cis': 0, 'trans': 9},
             'ENSG00000132305': {'cis': 0, 'trans': 6},
             'ENSG00000125637': {'cis': 0, 'trans': 2},
             'ENSG00000115233': {'cis': 0, 'trans': 1},
             'ENSG00000079156': {'cis': 0, 'trans': 2},
             'ENSG00000081320': {'cis': 0, 'trans': 2},
             'ENSG00000163719': {'cis': 0, 'trans': 3},
             'ENSG00000114650': {'cis': 0, 'trans': 2},
             'ENSG00000114268': {'cis': 0, 'trans': 3},
             'ENSG00000225697': {'cis': 0, 'trans': 3},
             'ENSG0000017747

In [ ]:
# gtex_results beta 0.02
# TOTAL: Found 3465 cis SNPs out of 3809034 read pairs

# gtex_results_cismask_mpi
# TOTAL: Found 6198 cis SNPs out of 3737195 read pairs

# cardio_results_cismask_mpi
# TOTAL: Found 7769 cis SNPs out of 193006 read pairs

# cardio_results_mono_us
# TOTAL: Found 1125 cis SNPs out of 50727 read pairs
